# C&R


In [ ]:

!pip install ultralytics

In [ ]:
from ultralytics import YOLO
model = YOLO('yolov8n.pt')

In [ ]:
res = model.predict('https://ultralytics.com/images/bus.jpg', save=True)

In [ ]:
res = model.predict('https://ultralytics.com/images/bus.jpg', save=True, conf=0.9)

In [ ]:
import cv2
from google.colab.patches import cv2_imshow
output_img = res[0].plot(labels=True, conf=True)
cv2_imshow(output_img)

In [ ]:
model.names

In [ ]:
res[0].boxes.cls

In [ ]:
res

In [ ]:
len(res)

In [ ]:
res[0].boxes


### 判定された物体の名前と個数を表示

In [ ]:
# 番号と物体の対応表を作る
cls =  res[0].names
# なにが何個出てきたのか数える
from collections import Counter
# 各要素の出現回数をカウント
counts = Counter(res[0].boxes.cls.tolist())
# 物体名を加えて表示
for number, count in counts.items():
    print(f"{number} は{cls[number]}で、個数は {count}個")


## 動画に映っている物体の判定

In [ ]:
!wget https://infoart.ait231.tokushima-u.ac.jp/images/pexels.mp4


In [ ]:
import cv2
cap = cv2.VideoCapture("/content/pixels.mp4")

In [ ]:
width = cap.get(cv2.CAP_PROP_FRAME_WIDTH)
print("動画の横幅", width)

height = cap.get(cv2.CAP_PROP_FRAME_HEIGHT)
print("動画の高さ", height)

frames = cap.get(cv2.CAP_PROP_FRAME_COUNT)
print("動画の総フレーム数", frames)

fps = cap.get(cv2.CAP_PROP_FPS)
print("１秒当たりのフレーム数:fps", fps)

print("動画の秒数", frames/fps)

In [ ]:
# ここから動画をフレーム単位で切り出し、その都度、
# モデルを適用して物体判定
num = 0
while cap.isOpened():
  # フレームごとに読み込む
  ret, img = cap.read()
  # フレームが残っている間、その物体判定を行う
  if ret:
    results = model(img, conf=0.5, verbose=False)
    # 映っている物体を確認
    categories = results[0].boxes.cls
    # 各要素の出現回数をカウント
    counts = Counter(results[0].boxes.cls.tolist())
    for number, count in counts.items():
      print(f"{num}フレームには{cat[number]} が{count} 個映っています")
    num = num + 1
    if num > frames:
      ret = None
  else :
    break

cap.release()

In [ ]:
Condition = True
while Condition:
        print("「ループ」")
        Condition = False

In [ ]:
# 自分が Google Colab にアップロードした動画を指定します
cap = cv2.VideoCapture("/content/pexels_videos_2880.mp4")

width = cap.get(cv2.CAP_PROP_FRAME_WIDTH)
height = cap.get(cv2.CAP_PROP_FRAME_HEIGHT)
frames = cap.get(cv2.CAP_PROP_FRAME_COUNT)
fps = cap.get(cv2.CAP_PROP_FPS)

# count = cap.get(cv2.CAP_PROP_FRAME_COUNT)
print("動画の総フレーム数", frames)

# 出力用に空の動画を作成しておく
writer = cv2.VideoWriter('./result.mp4',
                          cv2.VideoWriter_fourcc(*'MP4V',),fps,
                          frameSize=(int(width),int(height)))
num = 0
cls = model.names

# ここから動画をフレーム単位で切り出し、その都度、
# モデルを適用して物体判定
while cap.isOpened():
  # フレームごとに読み込む
  ret, img = cap.read()
  # フレームが残っている間、その物体判定を行う
  if ret:
    results = model(img, conf=0.5, verbose=False)
    # 特定の物体だけを判別したい場合 classesで、物体の番号を指定
    # 番号は上記を確認
    # results = model(img, conf=0.5, verbose=False, classes=[0,1,2])
    img = results[0].plot(labels=True, conf=True)
    writer.write(img)
    if num > count:
      ret = None
  else :
  　# 動画を保存する
    writer.release()
    break

cap.release()


## MediaPipeによるポーズ推定

In [ ]:
%%capture
!pip install mediapipe

In [ ]:
import cv2
import mediapipe as mp
from google.colab.patches import cv2_imshow

In [ ]:
!https://infoart.ait231.tokushima-u.ac.jp/images/golf.png

In [ ]:
image = cv2.imread("/content/golf.png")
cv2_imshow(image)

In [ ]:
# ポーズ推定の設定
mp_pose = mp.solutions.pose
pose = mp_pose.Pose(
    # 検出精度
    min_detection_confidence=0.5,
    # 静止画像であることを指定
    static_image_mode=True)
# 推定結果を画像に上書きするための準備
mp_drawing = mp.solutions.drawing_utils
# 元画像のコピーを用意
posed_img = image.copy()

# 読み込んだ画像から骨格を推定する
results = pose.process(image)

# 元画像のコピーに、ポーズ推定の結果を書き込む
mp_drawing.draw_landmarks(posed_img, results.pose_landmarks,
                              mp_pose.POSE_CONNECTIONS)

In [ ]:
cv2_imshow(posed_img)

In [ ]:
# ポーズ推定の設定
mp_pose = mp.solutions.pose
pose = mp_pose.Pose(
    # 検出精度
    min_detection_confidence=0.5,
    static_image_mode=True)
mp_drawing = mp.solutions.drawing_utils

# ここでRGB形式に変換したイメージを用意
rgb_image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
# 元画像から骨格を推定する
results = pose.process(image)
# ランドマークを書き込む
mp_drawing.draw_landmarks(posed_img, results.pose_landmarks,
                          mp_pose.POSE_CONNECTIONS)

In [ ]:

cv2_imshow(rgb_image)

In [ ]:
# posed_img をファイルとして保存する方法
cv2.imwrite('posed_image_rgb.jpg', posed_img)

In [ ]:
!https://infoart.ait231.tokushima-u.ac.jp/images/dalle3face.png

In [ ]:
image = cv2.imread("/content/dalle3face.png")
cv2_imshow(image)

In [ ]:
mp_face_mesh = mp.solutions.face_mesh

mp_drawing = mp.solutions.drawing_utils
drawing_spec = mp_drawing.DrawingSpec(thickness=1, circle_radius=1)
drawing_styles = mp.solutions.drawing_styles

face_mesh = mp_face_mesh.FaceMesh(static_image_mode=True, max_num_faces=1,
                                  refine_landmarks=True,
                                   min_detection_confidence=0.5)

# 元画像をコピー
face_img = image.copy()
# cv2を利用して読み込んでいるので顔判定するにあたりRGBに変換
rgb_image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

results = face_mesh.process(rgb_image)

# 推定結果を元画像に書き込む
for face_landmarks in results.multi_face_landmarks:
    mp_drawing.draw_landmarks(
    image= face_img,
    landmark_list = face_landmarks,
    landmark_drawing_spec = None,
    connections = mp_face_mesh.FACEMESH_CONTOURS,
    connection_drawing_spec = drawing_styles.get_default_face_mesh_contours_style())


In [ ]:
cv2_imshow(face_img)

In [ ]:
print(len(results.multi_face_landmarks[0].landmark))

In [ ]:
face_img = image.copy()
for face_landmarks in results.multi_face_landmarks:
    mp_drawing.draw_landmarks(
    image= face_img,
    landmark_list = face_landmarks,
    landmark_drawing_spec = None,
    # 以下2つの指定を変更
    connections = mp_face_mesh.FACEMESH_TESSELATION,
    connection_drawing_spec = drawing_styles.get_default_face_mesh_tesselation_style())

In [ ]:
cv2_imshow(face_img)

In [ ]:
face_img = image.copy()

for face_landmarks in results.multi_face_landmarks:
  mp_drawing.draw_landmarks(
    image= face_img,
    landmark_list = face_landmarks,
    landmark_drawing_spec = None,
    # 以下2つの指定を変更
    connections = mp_face_mesh.FACEMESH_IRISES,
    connection_drawing_spec = drawing_styles.get_default_face_mesh_iris_connections_style())

In [ ]:
cv2_imshow(face_img)

In [ ]:

face_img = image.copy()

for face_landmarks in results.multi_face_landmarks:

  mp_drawing.draw_landmarks(
    image= face_img,
    landmark_list = face_landmarks,
    landmark_drawing_spec = None,
    connections = mp_face_mesh.FACEMESH_CONTOURS,
    connection_drawing_spec = drawing_styles.get_default_face_mesh_contours_style())

  mp_drawing.draw_landmarks(
    image= face_img,
    landmark_list = face_landmarks,
    landmark_drawing_spec = None,
    connections = mp_face_mesh.FACEMESH_TESSELATION,
    connection_drawing_spec = drawing_styles.get_default_face_mesh_tesselation_style())

  mp_drawing.draw_landmarks(
    image= face_img,
    landmark_list = face_landmarks,
    landmark_drawing_spec = None,
    connections = mp_face_mesh.FACEMESH_IRISES,
    connection_drawing_spec = drawing_styles.get_default_face_mesh_iris_connections_style())

cv2_imshow(face_img)



### MVTec 異常検知サンプルデータ


## Azure Custom Vision APIの利用

In [ ]:
import requests
import json
# エンドポイントを指定*
url = "https://aicen*****nstance-prediction.cognitiveservices.azure.com/customvision/v3.0/Prediction/d13b88e0-8*******************ations/Iteration2/image"
# ヘッダーの設定
headers = {'content-type':'application/octet-stream', 'Prediction-Key':'xxxxxx'}
# リクエストを送る
response = requests.post(url, data=open("018.png","rb"), headers=headers)

In [ ]:
analysis = response.json()
name, pred = analysis["predictions"][0]["tagName"], analysis["predictions"][0]["probability"]
print(name, pred)
name, pred = analysis["predictions"][1]["tagName"], analysis["predictions"][1]["probability"]
print(name, pred)